In [7]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import models, layers

In [ ]:
import numpy as np

timesteps = 100
input_features = 32
output_features = 64

inputs = np.random.random((timesteps, input_features))

state_t = np.zeros((output_features))

W = np.random.random((output_features, input_features))
U = np.random.random((output_features, output_features))
b = np.random.random((output_features,))

successive_outputs = []
for input_t in inputs:
  output_t = np.tanh(np.dot(W, input_t) + np.dot(U, state_t) + b)

  successive_outputs.append(output_t)

  state_t = output_t

final_output_sequence = np.stack(successive_outputs, axis=0)

In [ ]:
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import models, layers

max_features = 10000
max_len = 500
batch_size = 32

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen = max_len)
x_test = sequence.pad_sequences(x_test, maxlen = max_len)

model = models.Sequential()
model.add(layers.Embedding(max_features, 32))
model.add(layers.SimpleRNN(32))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2, verbose=1)

17473536/17464789 [==============================] - 0s 0us/step
Epoch 1/10
157/157 [==============================] - 28s 169ms/step - loss: 0.5766 - acc: 0.6961 - val_loss: 0.4554 - val_acc: 0.7992
Epoch 2/10
157/157 [==============================] - 26s 166ms/step - loss: 0.3717 - acc: 0.8425 - val_loss: 0.4381 - val_acc: 0.7988
Epoch 3/10
157/157 [==============================] - 26s 165ms/step - loss: 0.2861 - acc: 0.8894 - val_loss: 0.4210 - val_acc: 0.8368
Epoch 4/10
157/157 [==============================] - 27s 170ms/step - loss: 0.2394 - acc: 0.9096 - val_loss: 0.3760 - val_acc: 0.8364
Epoch 5/10
157/157 [==============================] - 26s 166ms/step - loss: 0.1903 - acc: 0.9289 - val_loss: 0.3594 - val_acc: 0.8580
Epoch 6/10
157/157 [==============================] - 26s 166ms/step - loss: 0.1528 - acc: 0.9447 - val_loss: 0.5523 - val_acc: 0.8318
Epoch 7/10
157/157 [==============================] - 26s 165ms/step - loss: 0.1188 - acc: 0.9589 - val_loss: 0.4616 - val_ac

In [6]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras import models, layers
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)
x_train = sequence.pad_sequences(x_train, maxlen = 500)
x_test = sequence.pad_sequences(x_test, maxlen = 500)

model = models.Sequential()
model.add(layers.Embedding(10000, 32))
model.add(LSTM(32))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
157/157 [==============================] - 59s 356ms/step - loss: 0.5218 - acc: 0.7602 - val_loss: 0.3635 - val_acc: 0.8566
Epoch 2/10
157/157 [==============================] - 55s 350ms/step - loss: 0.3016 - acc: 0.8820 - val_loss: 0.2922 - val_acc: 0.8836
Epoch 3/10
157/157 [==============================] - 55s 351ms/step - loss: 0.2397 - acc: 0.9107 - val_loss: 0.3342 - val_acc: 0.8564
Epoch 4/10
157/157 [==============================] - 55s 351ms/step - loss: 0.2040 - acc: 0.9253 - val_loss: 0.3784 - val_acc: 0.8500
Epoch 5/10
157/157 [==============================] - 64s 409ms/step - loss: 0.1754 - acc: 0.9369 - val_loss: 0.4084 - val_acc: 0.8268
Epoch 6/10
157/157 [==============================] - 61s 387ms/step - loss: 0.1521 - acc: 0.9447 - val_loss: 0.3128 - val_acc: 0.8896
Epoch 7/10
157/157 [==============================] - 63s 404ms/step - loss: 0.1438 - acc: 0.9488 - val_loss: 0.3506 - val_acc: 0.8906
Epoch 8/10
157/157 [==============================] - 6

In [ ]:
import os

data_dir = 'path'
frame = os.path.join(data_dir, 'file')

f = open(frame)
data = f.read()
f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

float_data = np.zeros((len(lines), len(header)-1))
for i, line in enumerate(lines):
  values = [float(x) for x in line.split(',')[1:]]
  float_data[i, :] = values

mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std

In [ ]:
def generator(data, lookback, delay, min_index, max_index, shuffle=False, batch_size=128, step=6):
  if max_index is None:
    max_index = len(data) - delay - 1
  i = min_index + lookback
  while True:
    if shuffle:
      rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
    else:
      if i + batch_size >= max_index:
        i = min_index + lookback
      rows = np.arange(i, min(i + batch_size, max_index))
      i += len(rows)
    
    samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
    targets = np.zeros((len(rows),))

    for j, row in enumerate(rows):
      indices = range(rows[j] - lookback, rows[j], step)
      samples[j] = data[indices]
      targets[j] = data[rows[j] + delay][1]
    yield samples, targets

In [30]:
from tensorflow.keras import models, layers

model = models.Sequential()
model.add(layers.GRU(32, dropout=0.2, recurrent_dropout=0.2, input_shape=(None, 1)))
model.add(layers.Dense(1))
model.compile(optimizer='rmsprop', loss='mse')
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 32)                3360      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3,393
Trainable params: 3,393
Non-trainable params: 0
_________________________________________________________________


In [31]:
model = models.Sequential()
model.add(layers.GRU(32, dropout=0.1, recurrent_dropout=0.5, return_sequences=True, input_shape=(None, 1)))
model.add(layers.GRU(64, activation='relu', dropout=0.1, recurrent_dropout=0.5))
model.add(layers.Dense(1))
# model.compile(optimizer='rmsprop', loss='mse')
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 (None, None, 32)          3360      
                                                                 
 gru_2 (GRU)                 (None, 64)                18816     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 22,241
Trainable params: 22,241
Non-trainable params: 0
_________________________________________________________________


In [35]:
max_features = 10000
maxlen = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = [x[::-1] for x in x_train]
x_test = [x[::-1] for x in x_test]

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [36]:
model = models.Sequential()
model.add(layers.Embedding(max_features, 128))
model.add(layers.LSTM(32))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
histioy = model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
157/157 [==============================] - 79s 493ms/step - loss: 0.4851 - acc: 0.7695 - val_loss: 0.7316 - val_acc: 0.7072
Epoch 2/10
157/157 [==============================] - 77s 491ms/step - loss: 0.3157 - acc: 0.8783 - val_loss: 0.5856 - val_acc: 0.7768
Epoch 3/10
157/157 [==============================] - 78s 495ms/step - loss: 0.2594 - acc: 0.9051 - val_loss: 0.3155 - val_acc: 0.8842
Epoch 4/10
157/157 [==============================] - 78s 497ms/step - loss: 0.2141 - acc: 0.9240 - val_loss: 0.4036 - val_acc: 0.8430
Epoch 5/10
157/157 [==============================] - 85s 539ms/step - loss: 0.1906 - acc: 0.9327 - val_loss: 0.4366 - val_acc: 0.8194
Epoch 6/10
157/157 [==============================] - 88s 563ms/step - loss: 0.1756 - acc: 0.9392 - val_loss: 0.6261 - val_acc: 0.8070
Epoch 7/10
157/157 [==============================] - 78s 497ms/step - loss: 0.1546 - acc: 0.9474 - val_loss: 0.3987 - val_acc: 0.8748
Epoch 8/10
157/157 [==============================] - 7

In [38]:
model = models.Sequential()
model.add(layers.Embedding(max_features, 32))
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 32)          320000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               16640     
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 336,705
Trainable params: 336,705
Non-trainable params: 0
_________________________________________________________________


In [39]:
model = models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=maxlen))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 conv1d (Conv1D)             (None, 494, 32)           28704     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 98, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 92, 32)            7200      
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_7 (Dense)             (None, 1)                

In [40]:
model = models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=maxlen))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GRU(32, dropout=0.1, recurrent_dropout=0.5))
model.add(layers.Dense(1))

model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 conv1d_2 (Conv1D)           (None, 494, 32)           28704     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 98, 32)           0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 92, 32)            7200      
                                                                 
 gru_3 (GRU)                 (None, 32)                6336      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                      